# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-04-19 22:16:45] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=38554, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=781475039, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=Fals

[2025-04-19 22:16:58 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-19 22:16:58 TP0] Init torch distributed begin.


[2025-04-19 22:17:00 TP0] Init torch distributed ends. mem usage=0.21 GB
[2025-04-19 22:17:00 TP0] Load weight begin. avail mem=44.08 GB
[2025-04-19 22:17:00 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-19 22:17:02 TP0] Using model weights format ['*.safetensors']
[2025-04-19 22:17:02 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.76it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.75it/s]

[2025-04-19 22:17:02 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=41.25 GB, mem usage=2.83 GB.


...

[2025-04-19 22:17:05] INFO:     Started server process [270194]
[2025-04-19 22:17:05] INFO:     Waiting for application startup.
[2025-04-19 22:17:05] INFO:     Application startup complete.
[2025-04-19 22:17:05] INFO:     Uvicorn running on http://0.0.0.0:38554 (Press CTRL+C to quit)


[2025-04-19 22:17:05] INFO:     127.0.0.1:56958 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-19 22:17:06] INFO:     127.0.0.1:56962 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-19 22:17:06 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-19 22:17:10] INFO:     127.0.0.1:56964 - "POST /generate HTTP/1.1" 200 OK


[2025-04-19 22:17:10] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-04-19 22:17:10 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-19 22:17:10 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 6.55, #queue-req: 0, 


[2025-04-19 22:17:10 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 157.30, #queue-req: 0, 


[2025-04-19 22:17:11 TP0] Decode batch. #running-req: 1, #token: 121, token usage: 0.01, gen throughput (token/s): 157.32, #queue-req: 0, 
[2025-04-19 22:17:11] INFO:     127.0.0.1:56972 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-04-19 22:17:11] INFO:     127.0.0.1:56976 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-04-19 22:17:11] INFO:     127.0.0.1:56980 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-04-19 22:17:11 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-19 22:17:12] INFO:     127.0.0.1:56990 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-04-19 22:17:12] INFO:     127.0.0.1:57006 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-04-19 22:17:12] INFO:     127.0.0.1:57016 - "POST /flush_cache HTTP/1.1" 200 OK
[2025-04-19 22:17:12 TP0] Cache flushed successfully!


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-04-19 22:17:12] Start update_weights. Load format=auto
[2025-04-19 22:17:12 TP0] Update engine weights online from disk begin. avail mem=40.04 GB


[2025-04-19 22:17:12 TP0] Using model weights format ['*.safetensors']
[2025-04-19 22:17:12 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.38it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.38it/s]

[2025-04-19 22:17:13 TP0] Update weights end.
[2025-04-19 22:17:13 TP0] Cache flushed successfully!
[2025-04-19 22:17:13] INFO:     127.0.0.1:57032 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "meta-llama/Llama-3.2-1B-wrong"
    " (repository not found)."
)

[2025-04-19 22:17:13] Start update_weights. Load format=auto
[2025-04-19 22:17:13 TP0] Update engine weights online from disk begin. avail mem=40.04 GB
[2025-04-19 22:17:13 TP0] Failed to get weights iterator: meta-llama/Llama-3.2-1B-wrong (repository not found).
[2025-04-19 22:17:13] INFO:     127.0.0.1:52218 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [10]:
terminate_process(server_process)

embedding_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-19 22:17:13] Child process unexpectedly failed with an exit code 9. pid=271054


[2025-04-19 22:17:22] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=39291, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=6366721, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=Fals

[2025-04-19 22:17:31 TP0] Downcasting torch.float32 to torch.float16.


[2025-04-19 22:17:32 TP0] Overlap scheduler is disabled for embedding models.
[2025-04-19 22:17:32 TP0] Downcasting torch.float32 to torch.float16.
[2025-04-19 22:17:32 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-19 22:17:32 TP0] Init torch distributed begin.


[2025-04-19 22:17:32 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-19 22:17:32 TP0] Load weight begin. avail mem=62.16 GB


...

Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:10<00:00,  1.37s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:10<00:00,  1.57s/it]

[2025-04-19 22:17:44 TP0] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=47.72 GB, mem usage=14.43 GB.
[2025-04-19 22:17:44 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.55 GB, V size: 0.55 GB
[2025-04-19 22:17:44 TP0] Memory pool end. avail mem=46.35 GB


[2025-04-19 22:17:44 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-19 22:17:45] INFO:     Started server process [272893]
[2025-04-19 22:17:45] INFO:     Waiting for application startup.
[2025-04-19 22:17:45] INFO:     Application startup complete.
[2025-04-19 22:17:45] INFO:     Uvicorn running on http://0.0.0.0:39291 (Press CTRL+C to quit)
[2025-04-19 22:17:45] INFO:     127.0.0.1:46318 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-19 22:17:46] INFO:     127.0.0.1:46328 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-19 22:17:46 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-19 22:17:47] INFO:     127.0.0.1:46340 - "POST /encode HTTP/1.1" 200 OK
[2025-04-19 22:17:47] The server is fired up and ready to roll!


In [11]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-04-19 22:17:50 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-19 22:17:50] INFO:     127.0.0.1:46346 - "POST /encode HTTP/1.1" 200 OK


In [12]:
terminate_process(embedding_process)

[2025-04-19 22:17:50] Child process unexpectedly failed with an exit code 9. pid=273242
[2025-04-19 22:17:50] Child process unexpectedly failed with an exit code 9. pid=273175


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [13]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-19 22:17:59] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=39642, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=124864670, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=

[2025-04-19 22:18:09 TP0] Overlap scheduler is disabled for embedding models.
[2025-04-19 22:18:09 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-19 22:18:09 TP0] Init torch distributed begin.


[2025-04-19 22:18:09 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-19 22:18:09 TP0] Load weight begin. avail mem=59.91 GB


[2025-04-19 22:18:09 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-19 22:18:10 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

2025-04-19 22:18:14,495 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


[2025-04-19 22:18:14 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-19 22:18:15] INFO:     Started server process [274832]
[2025-04-19 22:18:15] INFO:     Waiting for application startup.
[2025-04-19 22:18:15] INFO:     Application startup complete.
[2025-04-19 22:18:15] INFO:     Uvicorn running on http://0.0.0.0:39642 (Press CTRL+C to quit)
[2025-04-19 22:18:15] INFO:     127.0.0.1:37948 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-19 22:18:16] INFO:     127.0.0.1:37956 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-19 22:18:16 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


2025-04-19 22:18:17,385 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90


In [14]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

2025-04-19 22:18:57,250 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-04-19 22:18:57,263 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-04-19 22:19:18,881 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-04-19 22:19:19 TP0] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-19 22:19:19] INFO:     127.0.0.1:37970 - "POST /encode HTTP/1.1" 200 OK
[2025-04-19 22:19:19] The server is fired up and ready to roll!
2025-04-19 22:19:19,105 - INFO - flashinfer.jit: Loading JIT ops: cascade


2025-04-19 22:19:33,781 - INFO - flashinfer.jit: Finished loading JIT ops: cascade
[2025-04-19 22:19:33] INFO:     127.0.0.1:37980 - "POST /classify HTTP/1.1" 200 OK


In [15]:
terminate_process(reward_process)

[2025-04-19 22:19:33] Child process unexpectedly failed with an exit code 9. pid=275167
[2025-04-19 22:19:33] Child process unexpectedly failed with an exit code 9. pid=275099


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [16]:
expert_record_server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-04-19 22:19:43] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=38077, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=49115969, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=0, show_t

[2025-04-19 22:19:54 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-19 22:19:54 TP0] Init torch distributed begin.


[2025-04-19 22:19:55 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-19 22:19:55 TP0] Load weight begin. avail mem=34.63 GB


[2025-04-19 22:19:55 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-19 22:19:55 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

[2025-04-19 22:20:03] INFO:     127.0.0.1:54396 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-19 22:20:03] INFO:     127.0.0.1:58834 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-19 22:20:03 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


2025-04-19 22:20:07,276 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-04-19 22:20:07,463 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False_sm90
2025-04-19 22:20:07,472 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


2025-04-19 22:20:07,575 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-04-19 22:20:08 TP0] Using default MoE config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1c-gpu-0/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can tune the config with https://github.com/sgl-project/sglang/blob/main/benchmark/kernels/fused_moe_triton/tuning_fused_moe_triton.py.


In [17]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

import glob

output_file = glob.glob("expert_distribution_*.csv")[0]
with open(output_file, "r") as f:
    print_highlight("\n| Layer ID | Expert ID | Count |")
    print_highlight("|----------|-----------|--------|")
    next(f)
    for i, line in enumerate(f):
        if i < 9:
            layer_id, expert_id, count = line.strip().split(",")
            print_highlight(f"| {layer_id:8} | {expert_id:9} | {count:6} |")

2025-04-19 22:20:08,825 - INFO - flashinfer.jit: Loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False
2025-04-19 22:20:08,997 - INFO - flashinfer.jit: Finished loading JIT ops: batch_decode_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False


[2025-04-19 22:20:09 TP0] Resetting expert distribution record...
[2025-04-19 22:20:09] INFO:     127.0.0.1:58848 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-04-19 22:20:09 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-04-19 22:20:09] INFO:     127.0.0.1:58852 - "POST /generate HTTP/1.1" 200 OK


[2025-04-19 22:20:09] INFO:     127.0.0.1:58860 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-04-19 22:20:09] INFO:     127.0.0.1:58842 - "POST /generate HTTP/1.1" 200 OK
[2025-04-19 22:20:09] The server is fired up and ready to roll!
[2025-04-19 22:20:09 TP0] Resetting expert distribution record...
[2025-04-19 22:20:09] INFO:     127.0.0.1:58874 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [18]:
terminate_process(expert_record_server_process)

[2025-04-19 22:20:09] Child process unexpectedly failed with an exit code 9. pid=280360


## Skip Tokenizer and Detokenizer

SGLang Runtime also supports skip tokenizer and detokenizer. This is useful in cases like integrating with RLHF workflow.

In [19]:
tokenizer_free_server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --skip-tokenizer-init
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-19 22:20:18] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=True, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=34996, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=908800339, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=Fal

[2025-04-19 22:20:33 TP0] Attention backend not set. Use fa3 backend by default.
[2025-04-19 22:20:33 TP0] Init torch distributed begin.


[2025-04-19 22:20:34 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-19 22:20:34 TP0] Load weight begin. avail mem=78.58 GB


[2025-04-19 22:20:35 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-19 22:20:36 TP0] Using model weights format ['*.safetensors']


...

Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.62it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.62it/s]

[2025-04-19 22:20:37 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=76.14 GB, mem usage=2.44 GB.


[2025-04-19 22:20:38 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.31 GB, V size: 0.31 GB
[2025-04-19 22:20:38 TP0] Memory pool end. avail mem=75.29 GB


[2025-04-19 22:20:38 TP0] 

CUDA Graph is DISABLED.
This will cause significant performance degradation.
CUDA Graph should almost never be disabled in most usage scenarios.
If you encounter OOM issues, please try setting --mem-fraction-static to a lower value (such as 0.8 or 0.7) instead of disabling CUDA Graph.

[2025-04-19 22:20:38 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-04-19 22:20:38] INFO:     Started server process [281676]
[2025-04-19 22:20:38] INFO:     Waiting for application startup.
[2025-04-19 22:20:38] INFO:     Application startup complete.
[2025-04-19 22:20:38] INFO:     Uvicorn running on http://127.0.0.1:34996 (Press CTRL+C to quit)


[2025-04-19 22:20:38] INFO:     127.0.0.1:44178 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-19 22:20:39] INFO:     127.0.0.1:44194 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-19 22:20:39 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

input_text = "What is the capital of France?"

input_tokens = tokenizer.encode(input_text)
print_highlight(f"Input Text: {input_text}")
print_highlight(f"Tokenized Input: {input_tokens}")

response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "input_ids": input_tokens,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 256,
            "stop_token_ids": [tokenizer.eos_token_id],
        },
        "stream": False,
    },
)
output = response.json()
output_tokens = output["output_ids"]

output_text = tokenizer.decode(output_tokens, skip_special_tokens=False)
print_highlight(f"Tokenized Output: {output_tokens}")
print_highlight(f"Decoded Output: {output_text}")
print_highlight(f"Output Text: {output['meta_info']['finish_reason']}")

[2025-04-19 22:20:44 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 
[2025-04-19 22:20:45] INFO:     127.0.0.1:44198 - "POST /generate HTTP/1.1" 200 OK
[2025-04-19 22:20:45] The server is fired up and ready to roll!


[2025-04-19 22:20:45 TP0] Decode batch. #running-req: 1, #token: 48, token usage: 0.00, gen throughput (token/s): 6.95, #queue-req: 0, 


[2025-04-19 22:20:45 TP0] Decode batch. #running-req: 1, #token: 88, token usage: 0.00, gen throughput (token/s): 195.31, #queue-req: 0, 
[2025-04-19 22:20:45] INFO:     127.0.0.1:51618 - "POST /generate HTTP/1.1" 200 OK


In [21]:
terminate_process(tokenizer_free_server_process)

[2025-04-19 22:20:45] Child process unexpectedly failed with an exit code 9. pid=282149
[2025-04-19 22:20:45] Child process unexpectedly failed with an exit code 9. pid=282082
